In [ ]:
import overpass
import json

In [ ]:
api = overpass.API(timeout=100)

In [ ]:
%%time
query = """
    [out:json][timeout:100];
    area[name=Berlin]->.searchHere;
    nwr[sport](area.searchHere);
    convert item ::id=id(),::=::,::geom=geom(),_osm_type=type();
    out geom;
"""
geometries = api.get(query, build=False)
geom_els = geometries["elements"]

query = """
    [out:json][timeout:100];
    area[name=Berlin]->.searchHere;
    nwr[sport](area.searchHere);
    convert item ::id=id(),::geom=geom(); 
    out center;
"""
centers = api.get(query, build=False)
center_els = centers["elements"]

In [ ]:
len(geom_els), len(center_els)

In [ ]:
def merge(geometries, centers):
    """ Merge center GeoJSON data into geometries as "center" key 
    
    Works inplace on the given geometries dict.
    """
    
    if centers["osm3s"] != geometries["osm3s"]:
        print("Different timestamps! Data might not match!")
        return
    gs = geometries["elements"]
    cs = centers["elements"]
    for i, c in enumerate(cs):
        if gs[i]["id"] != c["id"]:
            print(f'IDs at {i=} dont match up! {gs[i]["id"]} != {c["id"]}')
            return
        gs[i]["center"] = c["geometry"]
        
def set_type(response):
    for el in response["elements"]:
        if "_osm_type" not in el["tags"].keys():
            print("_osm_type Value missing!")
            return
        el["type"] = el["tags"]["_osm_type"]
        del el["tags"]["_osm_type"]

In [ ]:
merge(geometries, centers)
set_type(geometries)

In [ ]:
geometries["elements"][0]

In [ ]:
json.dump(response, open("sports.json", "w"))